In [1]:
import win32clipboard
import unicodedata
import csv

In [2]:
def get_clipboard_data():
    # get clipboard data
    win32clipboard.OpenClipboard()
    data = win32clipboard.GetClipboardData(win32clipboard.CF_UNICODETEXT)
    win32clipboard.CloseClipboard()
    
    return data.splitlines()

text = get_clipboard_data()
print(text)

['2 2⁄3 ounces dark chocolate', '5 tablespoons unsalted butter', '4 eggs, separated', '2⁄3 cup coconut sugar (or other natural sugar)', '1⁄2 cup hazelnut meal (or bleached hazelnuts ground in a food processor)', '1⁄3 cup unsweetened cocoa powder', '2 tablespoons brandy (optional) or 2 tablespoons rum (optional)', 'powdered sugar, for topping']


In [3]:
text = ['1⁄2 cup butter', '1 cup sugar', '2 eggs, beaten', '3 bananas, finely crushed (for serious and extreme moist and delicious, try 4 bananas)', '1 1⁄2 cups flour', '1 teaspoon baking soda', '1⁄2 teaspoon salt', '1⁄2 teaspoon vanilla']
print(text)

['1⁄2 cup butter', '1 cup sugar', '2 eggs, beaten', '3 bananas, finely crushed (for serious and extreme moist and delicious, try 4 bananas)', '1 1⁄2 cups flour', '1 teaspoon baking soda', '1⁄2 teaspoon salt', '1⁄2 teaspoon vanilla']


In [4]:
class RecipeConverter:
    def __init__(self):
        # Conversion table from volumetric to metric grams
        self.conversions = self.import_conversions('gram-conversions.csv')
                
    def import_conversions(self, filename):
        with open(filename) as csvfile:
            conversion_table = list(csv.reader(csvfile, delimiter=','))

            # Remove header
            conversion_table.pop(0)

            return conversion_table
        
    def parse_fractions(self, ingredient):
        # From Genius recipes
        ingredient = ingredient.replace('1⁄4', '.25')
        ingredient = ingredient.replace('1⁄3', '.33')
        ingredient = ingredient.replace('1⁄2', '.5')
        ingredient = ingredient.replace('2⁄3', '.66')
        ingredient = ingredient.replace('3⁄4', '.75')

        ingredient = ingredient.replace('1/8', '.125')
        ingredient = ingredient.replace('1/4', '.25')
        ingredient = ingredient.replace('1/3', '.33')
        ingredient = ingredient.replace('1/2', '.5')
        ingredient = ingredient.replace('2/3', '.66')
        ingredient = ingredient.replace('3/4', '.75')

        return ingredient

    def convert_ingredient(self, line, unit, conversion):
        # Extract number from measurement
        number_string = line.split(unit)[0].replace(' ','')
        number_float = float(number_string)

        # Convert butter cup to grams
        converted = number_float * float(conversion)

        # Construct the output ingredient line with original line
        line_out = '%.1f g%s'%(converted, line.split(unit)[1].strip('s'))
        return line_out

    def parse_recipe(self, text):
        output = []

        for recipe_line in text:
            flag_converted = False

            # For each recipe line item, look it up in the conversion table
            for ingredient_gram in self.conversions:
                if ingredient_gram[0] in recipe_line:
                    flag_converted = True

                    # Convert fractions to floats
                    line = self.parse_fractions(recipe_line)

                    if 'cup' in line:
                        output.append(self.convert_ingredient(line, 'cup', ingredient_gram[1]))
                        break
                    elif 'tablespoon' in line:
                        output.append(self.convert_ingredient(line, 'tablespoon', ingredient_gram[2]))
                        break
                    elif 'teaspoon' in line:
                        output.append(self.convert_ingredient(line, 'teaspoon', ingredient_gram[3]))
                        break
                    else:
                        flag_converted = False

            # If nothing was found, then use original line
            if not flag_converted:
                output.append(recipe_line)

        return output

rc = RecipeConverter()
rc.parse_recipe(text)

['113.5 g butter',
 '201.0 g sugar',
 '2 eggs, beaten',
 '3 bananas, finely crushed (for serious and extreme moist and delicious, try 4 bananas)',
 '180.0 g flour',
 '6.0 g baking soda',
 '2.5 g salt',
 '2.1 g vanilla']

In [5]:
def display_lines(text_list):
    for line in text_list:
        print(line)
        
    print()

In [6]:
text = get_clipboard_data()
print('***ORIGINAL***')
display_lines(text)

print('***CONVERTED***')
rc = RecipeConverter()
display_lines(rc.parse_recipe(text))

***ORIGINAL***
2 2⁄3 ounces dark chocolate
5 tablespoons unsalted butter
4 eggs, separated
2⁄3 cup coconut sugar (or other natural sugar)
1⁄2 cup hazelnut meal (or bleached hazelnuts ground in a food processor)
1⁄3 cup unsweetened cocoa powder
2 tablespoons brandy (optional) or 2 tablespoons rum (optional)
powdered sugar, for topping

***CONVERTED***
2 2⁄3 ounces dark chocolate
71.0 g unsalted butter
4 eggs, separated
132.7 g coconut sugar (or other natural sugar)
1⁄2 cup hazelnut meal (or bleached hazelnuts ground in a food processor)
28.1 g unsweetened cocoa powder
2 tablespoons brandy (optional) or 2 tablespoons rum (optional)
powdered sugar, for topping

